In [96]:
import datetime
import openai
import json
import inspect
import glob

from llmcam.ytlive import capture_youtube_live_frame
from llmcam.gpt4v import ask_gpt4v
from typing import Optional

In [97]:
def capture_youtube_live_frame_and_save() -> str:
    "Capture a jpeg file from YouTube Live and save in data directory"
    return str(capture_youtube_live_frame())

In [98]:
def ask_gpt4v_about_image_file(path:str) -> str:
    "Tell all about quantitative information from a given image file"
    info = ask_gpt4v(path)
    return json.dumps(info)

In [99]:
def tool_schema(func):
    """ automatically generate a schema from its parameters and docstring"""
    # Extract function name, docstring, and parameters
    func_name = func.__name__
    func_description = func.__doc__ or "No description provided."
    signature = inspect.signature(func)
    
    # Create parameters schema
    parameters_schema = {
        "type": "object",
        "properties": {},
        "required": []
    }
    
    # Populate properties and required fields
    for param_name, param in signature.parameters.items():
        param_type = param.annotation if param.annotation != inspect._empty else str
        # Map Python types to JSON Schema types
        json_type = "string" if param_type == str else "number" if param_type in [int, float] else "boolean"
        
        # Add parameter to schema
        parameters_schema["properties"][param_name] = {
            "type": json_type,
            "description": f"{param_name} parameter of type {param_type.__name__}"
        }
        
        # Mark as required if no default
        if param.default == inspect.Parameter.empty:
            parameters_schema["required"].append(param_name)
    
    # Build final tool schema
    tool_schema = [
        {
            "type": "function",
            "function": {
                "name": func_name,
                "description": func_description,
                "parameters": parameters_schema,
            }
        }
    ]
    return tool_schema

In [100]:
# Environmental setting up
tools = tool_schema(capture_youtube_live_frame_and_save) + tool_schema(ask_gpt4v_about_image_file)
messages = [{"role":"system", "content":"You are a helpful system administrator. Use the supplied tools to assist the user."}]

In [101]:
# Support functions to handle tool response,where res == response.choices[0].message
def fn_name(res): return res.tool_calls[0].function.name
def fn_args(res): return json.loads(res.tool_calls[0].function.arguments)    
def fn_exec(res): return globals().get(fn_name(res))(**fn_args(res))
def fn_result_content(res):
    """Create a content containing the result of the function call"""
    content = dict()
    content.update(fn_args(res))
    content.update({fn_name(res): fn_exec(res)})
    return json.dumps(content)

In [102]:
def complete(role, content, tool_call_id=None, printing=False):
    "Send completion request with messages, and save the response in messages again"
    messages.append({"role":role, "content":content, "tool_call_id":tool_call_id})
    response = openai.chat.completions.create(model="gpt-4o", messages=messages, tools=tools)
    res = response.choices[0].message
    messages.append(res.to_dict())
    if res.to_dict().get('tool_calls'):
        complete(role="tool", content=fn_result_content(res), tool_call_id=res.tool_calls[0].id)
    if printing==True:
        for i, m in enumerate(messages):
            print(i, m)

In [103]:
complete("user", "Hi, can you capture YouTube Live?", printing=True)

0 {'role': 'system', 'content': 'You are a helpful system administrator. Use the supplied tools to assist the user.'}
1 {'role': 'user', 'content': 'Hi, can you capture YouTube Live?', 'tool_call_id': None}
2 {'content': 'Yes, I can capture a frame from a YouTube Live stream. Would you like me to proceed with capturing one?', 'refusal': None, 'role': 'assistant'}


In [104]:
complete("user", "Yes, please", printing=True)

[youtube] Extracting URL: https://www.youtube.com/watch?v=LMZQ7eFhm58
[youtube] LMZQ7eFhm58: Downloading webpage
[youtube] LMZQ7eFhm58: Downloading ios player API JSON
[youtube] LMZQ7eFhm58: Downloading web creator player API JSON
[youtube] LMZQ7eFhm58: Downloading m3u8 information
31.10.2024 22:15:10 Presidentinlinna
0 {'role': 'system', 'content': 'You are a helpful system administrator. Use the supplied tools to assist the user.'}
1 {'role': 'user', 'content': 'Hi, can you capture YouTube Live?', 'tool_call_id': None}
2 {'content': 'Yes, I can capture a frame from a YouTube Live stream. Would you like me to proceed with capturing one?', 'refusal': None, 'role': 'assistant'}
3 {'role': 'user', 'content': 'Yes, please', 'tool_call_id': None}
4 {'content': None, 'refusal': None, 'role': 'assistant', 'tool_calls': [{'id': 'call_Av8BEHRQLU5vOZ9xtOsXSTWc', 'function': {'arguments': '{}', 'name': 'capture_youtube_live_frame_and_save'}, 'type': 'function'}]}
5 {'role': 'tool', 'content': '{

In [105]:
files = sorted(glob.glob("./data/cap_*.jpg"))
len(files), files[:3]

(496,
 ['./data/cap_2024.09.28_15:59:06_Presidentinlinna.jpg',
  './data/cap_2024.09.28_16:00:11_Presidentinlinna.jpg',
  './data/cap_2024.09.28_16:01:16_Etelasatama.jpg'])

In [106]:
complete("user", f"Hi, can you tell any infomation about {files[-1]}?", printing=True) 

0 {'role': 'system', 'content': 'You are a helpful system administrator. Use the supplied tools to assist the user.'}
1 {'role': 'user', 'content': 'Hi, can you capture YouTube Live?', 'tool_call_id': None}
2 {'content': 'Yes, I can capture a frame from a YouTube Live stream. Would you like me to proceed with capturing one?', 'refusal': None, 'role': 'assistant'}
3 {'role': 'user', 'content': 'Yes, please', 'tool_call_id': None}
4 {'content': None, 'refusal': None, 'role': 'assistant', 'tool_calls': [{'id': 'call_Av8BEHRQLU5vOZ9xtOsXSTWc', 'function': {'arguments': '{}', 'name': 'capture_youtube_live_frame_and_save'}, 'type': 'function'}]}
5 {'role': 'tool', 'content': '{"capture_youtube_live_frame_and_save": "../data/cap_2024.10.31_22:15:10_Presidentinlinna.jpg"}', 'tool_call_id': 'call_Av8BEHRQLU5vOZ9xtOsXSTWc'}
6 {'content': 'I captured a frame from the YouTube Live stream and saved it as **Presidentinlinna.jpg** in the data directory. If you need any more information about the imag

In [107]:
print(messages[-1]['content'])

The image captured at **Valkosaari** on **October 31, 2024, at 11:52:57 AM** contains the following details:

- **Dimensions**: 1280x720 pixels
- **Buildings**: There are 4 buildings visible, ranging from 1 to 2 stories in height.
- **Vehicles**: No vehicles are visible in the image.
- **Water Bodies**: There's a visible sea, but no boats are present.
- **Street Lights**: There are no street lights visible.
- **People**: No people are seen in the image.
- **Lighting**: Taken in the morning with no artificial lighting. The lighting conditions suggest an overcast sky.
- **Visibility**: The visibility is clear.
- **Sky**: The sky is visible with overcast light conditions.

If you need further analysis or have other questions, let me know!


In [108]:
content = f"""
Here's a list of files, {",".join(files)}. The format is "cap_%Y.%m.%d_%H:%M:%S_<place>.
please list up all places
"""
complete("user", content)

In [109]:
messages[-1]

{'content': 'The list of places found in the file names you provided is as follows:\n\n1. Presidentinlinna\n2. Etelasatama\n3. Olympiaterminaali\n4. Torni\n5. Tuomiokirkko\n6. Kauppatori\n7. Valkosaari\n\nIf you need detailed analysis for specific images from these locations, please let me know!',
 'refusal': None,
 'role': 'assistant'}

In [110]:
complete("user", "On which dates were Tuomikirkko captured?")

In [111]:
messages[-1]

{'content': 'Tuomiokirkko was captured on the following dates:\n\n- 2024.09.28\n- 2024.09.29\n- 2024.09.30\n- 2024.10.04\n- 2024.10.05\n- 2024.10.06\n- 2024.10.07\n- 2024.10.28\n- 2024.10.29\n- 2024.10.30\n- 2024.10.31',
 'refusal': None,
 'role': 'assistant'}

In [112]:
complete("user", "Tell me the latest status of Tuomikirkko at captured?")

In [113]:
messages[-1]

{'content': 'The latest capture of Tuomiokirkko shows the following details:\n\n- **Timestamp**: October 31, 2024, at 11:44:47 AM\n- **Dimensions**: 1280x720 pixels\n- **Buildings**: There are 20 buildings, ranging from 2 to 10 stories in height.\n- **Vehicles**: No vehicles are visible in the image.\n- **Water Bodies**: None visible.\n- **Street Lights**: There are no street lights visible.\n- **People**: No people are seen in the image.\n- **Lighting**: Captured during the day with no artificial lighting.\n- **Visibility**: The visibility is clear.\n- **Sky**: The sky is visible with daylight conditions.\n\nIf you need more information or assistance, feel free to ask!',
 'refusal': None,
 'role': 'assistant'}

In [114]:
complete("user", "Please continue to capture every one minute till we'll get the one for Tuomiokirkko")
messages[-1]

[youtube] Extracting URL: https://www.youtube.com/watch?v=LMZQ7eFhm58
[youtube] LMZQ7eFhm58: Downloading webpage
[youtube] LMZQ7eFhm58: Downloading ios player API JSON
[youtube] LMZQ7eFhm58: Downloading web creator player API JSON
[youtube] LMZQ7eFhm58: Downloading m3u8 information
31.10.2024 22:15:45 Presidentinlinna
[youtube] Extracting URL: https://www.youtube.com/watch?v=LMZQ7eFhm58
[youtube] LMZQ7eFhm58: Downloading webpage
[youtube] LMZQ7eFhm58: Downloading ios player API JSON
[youtube] LMZQ7eFhm58: Downloading web creator player API JSON
[youtube] LMZQ7eFhm58: Downloading m3u8 information
31.10.2024 22:15:50 Presidentinlinna
[youtube] Extracting URL: https://www.youtube.com/watch?v=LMZQ7eFhm58
[youtube] LMZQ7eFhm58: Downloading webpage
[youtube] LMZQ7eFhm58: Downloading ios player API JSON
[youtube] LMZQ7eFhm58: Downloading web creator player API JSON
[youtube] LMZQ7eFhm58: Downloading m3u8 information
31.10.2024 22:15:50 Presidentinlinna
[youtube] Extracting URL: https://www.yo

{'content': "The latest capture is still from Presidentinlinna. I'll continue to capture frames periodically until we get one from Tuomiokirkko. Please hold on.",
 'refusal': None,
 'role': 'assistant'}

In [115]:
complete("user", "tell me about that capture, then")
messages[-1]

FileNotFoundError: [Errno 2] No such file or directory: '../data/cap_2024.10.31_22:16:00_Presidentinlinna.jpg'